## 양돈 월별 데이터의 Database 생성
* database명: pig_db

In [24]:
import mysql.connector

conn = mysql.connector.connect(
    host = 'database-1.cba44miw26by.ap-northeast-2.rds.amazonaws.com',
    port = 3306,
    user = 'kjy',
    password = '0000',
    database = 'pig'
)

cursor = conn.cursor(buffered=True)

In [11]:
import pandas as pd

raw_data = pd.read_excel('양돈_월별_번식통계.xlsx', header=1)
raw_data = raw_data.drop(columns=['지역'])
raw_data.head()

/home/jykim/venv/eda_venv/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,년도,규모,농장,데이터구분,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
0,2020,100~200,PIGGO_156,분만율,81.3,81.1,73.9,76.6,80.6,68.6,79.4,77.4,75.9,79.4,72.2,75.0
1,2019,100~200,PIGGO_156,평균이유,9.0,8.5,9.9,9.0,9.1,9.0,9.1,9.7,8.6,9.0,9.5,10.1
2,2018,100~200,PIGGO_156,임신사고(불규칙),0.0,0.0,2.0,3.0,1.0,1.0,2.0,3.0,4.0,2.0,4.0,2.0
3,2017,100~200,PIGGO_156,7일내재귀율,87.0,83.3,87.5,78.9,78.9,90.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2017,100~200,PIGGO_156,임신사고(도태),NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
import numpy as np

raw_data = raw_data.replace({np.nan: None})
raw_data.head()

,년도,규모,농장,데이터구분,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
0,2020,100~200,PIGGO_156,분만율,81.3,81.1,73.9,76.6,80.6,68.6,79.4,77.4,75.9,79.4,72.2,75.0
1,2019,100~200,PIGGO_156,평균이유,9.0,8.5,9.9,9.0,9.1,9.0,9.1,9.7,8.6,9.0,9.5,10.1
2,2018,100~200,PIGGO_156,임신사고(불규칙),0.0,0.0,2.0,3.0,1.0,1.0,2.0,3.0,4.0,2.0,4.0,2.0
3,2017,100~200,PIGGO_156,7일내재귀율,87.0,83.3,87.5,78.9,78.9,90.0,None,None,None,None,None,None
4,2017,100~200,PIGGO_156,임신사고(도태),None,None,None,None,None,None,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
len(raw_data)

12849

In [26]:
cursor.execute(
    '''create table pig_db (
    id int not null,
    년도 int,
    규모 varchar(16),
    농장 varchar(16),
    데이터구분 varchar(16),
    1월 float,
    2월 float,
    3월 float,
    4월 float,
    5월 float,
    6월 float,
    7월 float,
    8월 float,
    9월 float,
    10월 float,
    11월 float,
    12월 float
    )'''
)


In [34]:
sql = '''insert into pig_db values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
for i, row in raw_data.iterrows():
    data = (i,) + tuple(row)
    cursor.execute(sql, data)
    if i < 5:
        print(tuple(data))

conn.commit()

(0, 2020, '100~200', 'PIGGO_156', '분만율', 81.3, 81.1, 73.9, 76.6, 80.6, 68.6, 79.4, 77.4, 75.9, 79.4, 72.2, 75.0)
(1, 2019, '100~200', 'PIGGO_156', '평균이유', 9.0, 8.5, 9.9, 9.0, 9.1, 9.0, 9.1, 9.7, 8.6, 9.0, 9.5, 10.1)
(2, 2018, '100~200', 'PIGGO_156', '임신사고(불규칙)', 0.0, 0.0, 2.0, 3.0, 1.0, 1.0, 2.0, 3.0, 4.0, 2.0, 4.0, 2.0)
(3, 2017, '100~200', 'PIGGO_156', '7일내재귀율', 87.0, 83.3, 87.5, 78.9, 78.9, 90.0, None, None, None, None, None, None)
(4, 2017, '100~200', 'PIGGO_156', '임신사고(도태)', None, None, None, None, None, None, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)


In [36]:
cursor.execute('''select count(*) from pig_db''')
result = cursor.fetchall()
print(result)

[(12849,)]


In [42]:
cols = ['id'] + list(raw_data.columns)
print(cols)

['id', '년도', '규모', '농장', '데이터구분', '1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월']


In [45]:
conn.close()

#### === 백업 완료 ===

## 데이터 분석

### 1. 

In [50]:
import mysql.connector

conn = mysql.connector.connect(
    host = 'database-1.cba44miw26by.ap-northeast-2.rds.amazonaws.com',
    port = 3306,
    user = 'kjy',
    password = '0000',
    database = 'pig'
)

cursor = conn.cursor(buffered=True)

In [51]:
cursor.execute('select * from pig_db')
result = cursor.fetchall()
df = pd.DataFrame(result)
df.columns = cols
df.head()


,id,년도,규모,농장,데이터구분,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
0,0,2020,100~200,PIGGO_156,분만율,81.3,81.1,73.9,76.6,80.6,68.6,79.4,77.4,75.9,79.4,72.2,75.0
1,1,2019,100~200,PIGGO_156,평균이유,9.0,8.5,9.9,9.0,9.1,9.0,9.1,9.7,8.6,9.0,9.5,10.1
2,2,2018,100~200,PIGGO_156,임신사고(불규칙),0.0,0.0,2.0,3.0,1.0,1.0,2.0,3.0,4.0,2.0,4.0,2.0
3,3,2017,100~200,PIGGO_156,7일내재귀율,87.0,83.3,87.5,78.9,78.9,90.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2017,100~200,PIGGO_156,임신사고(도태),NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
cursor.execute('''select distinct 데이터구분 from pig_db''')
result = cursor.fetchall()
print(result)

[('분만율',), ('평균이유',), ('임신사고(불규칙)',), ('7일내재귀율',), ('임신사고(도태)',), ('모돈회전율',), ('임신사고(공태)',), ('임신사고(2차)',), ('임신사고(조기)',), ('미라합계',), ('임신사고(불임)',), ('총산합계',), ('평균총산',), ('사산합계',), ('임신사고(유산)',), ('임신사고(지연)',), ('임신사고(1차)',), ('임신사고(폐사)',), ('평균실산',)]


In [55]:
distinct = [x[0] for x in result]
print(distinct)

['분만율', '평균이유', '임신사고(불규칙)', '7일내재귀율', '임신사고(도태)', '모돈회전율', '임신사고(공태)', '임신사고(2차)', '임신사고(조기)', '미라합계', '임신사고(불임)', '총산합계', '평균총산', '사산합계', '임신사고(유산)', '임신사고(지연)', '임신사고(1차)', '임신사고(폐사)', '평균실산']
